<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess human brain single-cell data from Garcia 2022 Nature  
Publication: https://www.nature.com/articles/s41586-022-04521-7

Also see https://nsun.shinyapps.io/scbbb/ (only showing ex-vivo dataset)

In [ ]:
import datetime
datetime.date.today().strftime('%d/%m/%Y')

In [ ]:
import os
os.getlogin()

In [ ]:
import sys
import fnmatch 

In [ ]:
import anndata
import scanpy as sc
import scipy as sci
#sc.logging.print_versions()

In [ ]:
sc.settings.verbosity = 3

In [ ]:
#import loompy

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

In [ ]:
from matplotlib.pyplot import rc_context

In [ ]:
os.getcwd()

In [ ]:
main_dir='/run/user/1000/gvfs/smb-share:server=138.245.4.35,share=bd-dichgans/SF' # Linux
main_dir='\\\isdsynnas.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win

In [ ]:
main_dir='/cluster2/sfrerich/jupyter/p6-foxf2-per-celltype/data' # HPC

# Load data

Downloaded from http://compbio.mit.edu/scBBB/ (27/02/2023)

In [ ]:
# Turn rds to tsv on HPC
# /opt/R/4.0.0/bin/R
# library(data.table)
# 
# main <- "/cluster2/sfrerich/jupyter/p6-foxf2-per-celltype/data/P6_vasc_scRNAseq/"
# 
# f <- readRDS(paste0(main, 'Garcia2022/preprocessed/brain.BBB.human.counts.rds'))
# f <- readRDS(paste0(main, 'Garcia2022/brain.BBB.human.counts.GEOsubmission.rds'))
# f <- readRDS(paste0(main, 'Garcia2022/ROSMAP.VascularCells.counts.rds'))
# m <- readRDS(paste0(main, 'Garcia2022/brain.BBB.human.meta.GEOsubmission.rds')) # export directly
# 
# fd <- as.data.table(as.matrix(f))
# rownames(fd) <- as.data.frame(f@Dimnames[1])[,1] # dense
# colnames(fd) <- as.data.frame(f@Dimnames[2])[,1]
# 
# fwrite(fd, paste0(main, 'Garcia2022/preprocessed/brain.BBB.human.counts.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(fd, paste0(main, 'Garcia2022/brain.BBB.human.counts.GEOsubmission.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(fd, paste0(main, 'Garcia2022/ROSMAP.VascularCells.counts.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(m, paste0(main, 'Garcia2022/brain.BBB.human.meta.GEOsubmission.txt'), sep="\t", col.names=TRUE)

In [ ]:
# from base folder
# not annotated / not used

In [ ]:
# ad = sc.read_csv(main_dir+"/P6_vasc_scRNAseq/Garcia2022/brain.BBB.human.counts.GEOsubmission.tsv", delimiter="\t").transpose()

In [ ]:
#ad

In [ ]:
# meta = pd.read_csv(main_dir+'/P6_vasc_scRNAseq/Garcia2022/brain.BBB.human.meta.GEOsubmission.txt', sep='\t')
# meta.head(2)

## ROSMAP dataset

Nonpathological post-mortem brains

In [ ]:
# from base folder: ROSMAP study / annotated, vascular cells only

In [ ]:
main_dir

In [ ]:
ad_ROSMAP = sc.read_csv(main_dir+"/P06_vasc_scRNAseq/Garcia2022/ROSMAP.VascularCells.counts.tsv", delimiter="\t").transpose()


In [ ]:
ad_ROSMAP.obs_names_make_unique()
ad_ROSMAP.var_names_make_unique()

In [ ]:
ad_ROSMAP

In [ ]:
ad_ROSMAP.obs.head()

In [ ]:
meta_ROSMAP = pd.read_csv(main_dir+'/P06_vasc_scRNAseq/Garcia2022/ROSMAP.VascularCells.meta.txt', sep='\t')
meta_ROSMAP.head(2)

In [ ]:
pd.value_counts(meta_ROSMAP.celltype)

In [ ]:
pd.value_counts(meta_ROSMAP.subtype)

In [ ]:
ad_ROSMAP.obs = ad_ROSMAP.obs.join(meta_ROSMAP)

Check if data is normalized:

In [ ]:
ad_ROSMAP.layers

In [ ]:
ad_ROSMAP.X

In [ ]:
# show expression of 100 random genes (across all spots)
import random
import seaborn as sns
random_genes=random.sample(range(0, ad_ROSMAP.X.shape[1]), 100)
adata_sub = ad_ROSMAP[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007))
sns.set_theme(style='white')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_theme(style='white')
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

Data is not normalized, therefore normalize and log-transform

In [ ]:
ad_ROSMAP.layers["counts"] = ad_ROSMAP.X.copy() # save unnormalized raw RNA counts - retrieve via ad.X = ad.layers["counts"]

In [ ]:
sc.pp.normalize_total(ad_ROSMAP, inplace=True) # Normalize each spot by total counts over all genes, so that every cell has the same total count after normalization.

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = ad_ROSMAP[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005))
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10))
plt.show()

In [ ]:
sc.pp.log1p(ad_ROSMAP) # X = log(X + 1)

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = ad_ROSMAP[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.5) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005));
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)));
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
ad_ROSMAP.layers["normalized"] = ad_ROSMAP.X.copy() # retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(ad_ROSMAP)
sc.pl.highly_variable_genes(ad_ROSMAP)

In [ ]:
ad_ROSMAP

In [ ]:
ad_ROSMAP.layers

In [ ]:
pd.value_counts(meta_ROSMAP.celltype)

In [ ]:
# Renaming
old_to_new = {'aEndo':'aECs',
              'capEndo':'capECs',
              'vEndo':'vECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Per3':'Pericytes',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs'}
ad_ROSMAP.obs['clusters2'] = (ad_ROSMAP.obs['subtype'].map(old_to_new).astype('category'))

In [ ]:
# Renaming
old_to_new = {'Endo':'ECs',
              'Per':'Pericytes',
              'Fib':'Fibroblasts',
              'SMC':'SMCs'}
ad_ROSMAP.obs['clusters'] = (ad_ROSMAP.obs['celltype'].map(old_to_new).astype('category'))

### Dim Reduction

Nonpathological post-mortem brains

In [ ]:
sc.tl.pca(ad_ROSMAP)
sc.pl.pca_variance_ratio(ad_ROSMAP)

In [ ]:
sc.pp.neighbors(ad_ROSMAP)
sc.tl.umap(ad_ROSMAP)

In [ ]:
sc.pl.umap(ad_ROSMAP, color=['clusters','clusters2'], wspace=0.5)

In [ ]:
ad_ROSMAP.obs.clusters2.value_counts()

### Marker genes

In [ ]:
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(ad_ROSMAP,n_genes=7)

### Vascular marker genes

In [ ]:
# Basic QC metrics
ad_ROSMAP.var['mt'] = ad_ROSMAP.var_names.str.startswith('MT-') 
sc.pp.calculate_qc_metrics(ad_ROSMAP, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(ad_ROSMAP, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],jitter=0.4, multi_panel=True)

In [ ]:
# DEA requires HPC
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(ad_ROSMAP, n_genes=20, sharey=False)

Export DEG table

In [ ]:
# copied for HPC / usually see p6_helper_functions.py
# extract genes from previous DE analysis (sc.tl.rank_genes_groups())
def rank_genes_groups_df(adata, group, adj_pval_cutoff : float = None, log2fc_cutoff = None, sortby : int = 'zscore'): 
        # sortby = 'zscore' shows only upregulated genes, sorted by z-score (default from rank_genes_groups)
        # to show all DE genes (up and down), set to 'zscore_abs'
        # to sort by ['pvals_adj','log2FC'] (up and down) set to 'pval_adj'
    d = pd.DataFrame() 
    for k in ['scores', 'names', 'logfoldchanges', 'pvals', 'pvals_adj']: 
        d[k] = adata.uns["rank_genes_groups"][k][group] 
    if adj_pval_cutoff is not None: 
        d = d[d["pvals_adj"] < adj_pval_cutoff] 
    if log2fc_cutoff is not None: 
        d = d[d["logfoldchanges"].abs() > log2fc_cutoff] 
    # format table
    d=d.rename(columns={"names": "gene", "logfoldchanges": "log2FC", "scores": "zscore"})  # as defined in sc.tl.rank_genes_groups
    d.index = d['gene']
    d.index.name = None # remove index name
    d['FC'] = 2**d['log2FC']
    d['nlog10_pval_adj'] = np.log10(d['pvals_adj']) * (-1) # consider excel compatibility for naming
    # sort
    if sortby == 'zscore':
        d=d.sort_values(by=['zscore'], ascending=False)
    elif sortby == 'zscore_abs':
        d=d.sort_values(by=['zscore'], key=abs, ascending=False)
    elif sortby == 'pval_adj':
        d=d.sort_values(by=['pvals_adj','log2FC'], key=abs, ascending=[True,False])
    d['gene'] = d['gene'].str.upper()
    d['group'] = group
    d = d[['gene','log2FC','FC','pvals','pvals_adj','nlog10_pval_adj','zscore','group']] # oder columns
    return d

In [ ]:
# export DEG table
celltypes = ad_ROSMAP.obs.clusters.unique().tolist()
merged_df = pd.DataFrame()
for group in celltypes:
    rank_df = rank_genes_groups_df(adata=ad_ROSMAP, group=group, adj_pval_cutoff=None, log2fc_cutoff=None, sortby='zscore')
    merged_df = pd.concat([merged_df, rank_df])
    merged_df['reference'] = 'all other clusters'
merged_df.head(3)

In [ ]:
merged_df.group.unique().tolist()

In [ ]:
# export as csv
merged_df.to_csv(main_dir+"//P06_vasc_scRNAseq//Garcia2022//DEA//"+ date.today().strftime("%Y%m%d")+'_DE_genes_Garcia2022.csv',sep=';')

In [ ]:
# DEA requires HPC
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters2', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(ad_ROSMAP, n_genes=20, sharey=False)

Export DEG table

In [ ]:
# export DEG table
celltypes = ad_ROSMAP.obs.clusters2.unique().tolist()
merged_df = pd.DataFrame()
for group in celltypes:
    rank_df = rank_genes_groups_df(adata=ad_ROSMAP, group=group, adj_pval_cutoff=None, log2fc_cutoff=None, sortby='zscore')
    merged_df = pd.concat([merged_df, rank_df])
    merged_df['reference'] = 'all other clusters'
merged_df.head(3)

In [ ]:
merged_df.group.unique().tolist()

In [ ]:
# export as csv
merged_df.to_csv(main_dir+"//P06_vasc_scRNAseq//Garcia2022//DEA//"+ date.today().strftime("%Y%m%d")+'_DE_genes_Garcia2022_incl_zonation.csv',sep=';')

### Focus on: Foxf2

In [ ]:
gene="FOXF2"

In [ ]:
sc.pl.dotplot(ad_ROSMAP, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_ROSMAP, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Ex-vivo dataset

Healthy tissue from epilepsy patients

In [ ]:
# from preprocessed folder:

In [ ]:
ad = sc.read_csv(main_dir+"/P06_vasc_scRNAseq/Garcia2022/preprocessed/brain.BBB.human.counts.tsv", delimiter="\t").transpose()

In [ ]:
ad.obs_names_make_unique()
ad.var_names_make_unique()

In [ ]:
ad

In [ ]:
ad.var.head()

In [ ]:
ad.obs.head()

In [ ]:
# Add metadata

In [ ]:
meta = pd.read_csv(main_dir+'/P06_vasc_scRNAseq/Garcia2022/preprocessed/brain.BBB.human.meta.txt', sep='\t')
meta.head(2)

In [ ]:
meta.shape

In [ ]:
meta2 = pd.read_csv(main_dir+'/P06_vasc_scRNAseq/Garcia2022/preprocessed/brain.BBB.human.vascular.final.Jan2022.metadata.txt', sep='\t')
meta2.index = meta2["orig.ident"]
meta2.index.name = None
meta2.head(2)

In [ ]:
meta2.shape

In [ ]:
pd.value_counts(meta2.cellsubtype)

In [ ]:
meta = meta.join(meta2[["cellsubtype"]])

In [ ]:
meta['clusters'] = meta['cellsubtype'].fillna(meta['celltype'])

In [ ]:
meta.head(2)

In [ ]:
ad.obs = ad.obs.join(meta)

In [ ]:
ad.obs.columns

In [ ]:
ad.obs.head(2)

In [ ]:
pd.value_counts(ad.obs["snRNAPreparation"])

In [ ]:
pd.value_counts(meta.SampleID)

In [ ]:
pd.value_counts(ad.obs.PatientID)

In [ ]:
pd.value_counts(ad.obs.celltype)

In [ ]:
pd.value_counts(ad.obs.cellsubtype)

In [ ]:
pd.value_counts(ad.obs.clusters)

In [ ]:
# Renaming
old_to_new = {'aEndo':'aECs',
              'Astro':'Astrocytes',
              'capEndo':'capECs',
              'vEndo':'vECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Microglia':'Microglia',
              'Ex':'Neurons',
              'In':'Neurons',
              'OPC':'OPCs',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs',
              'Oligo':'Oligos'}
ad.obs['clusters2'] = (ad.obs['clusters'].map(old_to_new).astype('category'))

In [ ]:
# Renaming
old_to_new = {'aEndo':'ECs',
              'Astro':'Astrocytes',
              'capEndo':'ECs',
              'vEndo':'ECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Microglia':'Microglia',
              'Ex':'Neurons',
              'In':'Neurons',
              'OPC':'OPCs',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs',
              'Oligo':'Oligos'}
ad.obs['clusters'] = (ad.obs['clusters'].map(old_to_new).astype('category'))

In [ ]:
adata=ad

In [ ]:
del ad

Check if data is normalized:

In [ ]:
adata.layers

In [ ]:
adata.X

In [ ]:
# show expression of 100 random genes (across all spots)
import random
import seaborn as sns
random_genes=random.sample(range(0, adata.X.shape[1]), 100)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007))
sns.set_theme(style='white')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_theme(style='white')
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

Data is not normalized, therefore normalize and log-transform

In [ ]:
adata.layers["counts"] = adata.X.copy() # save unnormalized raw RNA counts - retrieve via ad.X = ad.layers["counts"]

In [ ]:
sc.pp.normalize_total(adata, inplace=True) # Normalize each spot by total counts over all genes, so that every cell has the same total count after normalization.

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005))
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10))
plt.show()

In [ ]:
sc.pp.log1p(adata) # X = log(X + 1)

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.5) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005));
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)));
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
adata.layers["normalized"] = adata.X.copy() # retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(adata)
sc.pl.highly_variable_genes(adata)

In [ ]:
adata

In [ ]:
adata.layers

### Dim Reduction

In [ ]:
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata)

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='clusters')

### Marker genes

In [ ]:
sc.tl.rank_genes_groups(adata, 'clusters')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata,n_genes=5)

### Focus on: Foxf2

In [ ]:
gene="FOXF2"

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Merge

In [ ]:
adata = anndata.concat([adata, ad_ROSMAP], join="inner") # ignore batch effects etc because data is pre-annotated

In [ ]:
adata

# Focus on: Foxf2

In [ ]:
gene="FOXF2"

## Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Foxf2_summarized/'+ date.today().strftime("%Y%m%d")+'_'+name+'_normalized.csv',sep=';')

## Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Foxf2_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+name+'_incl_zonation_normalized.csv',sep=';')

# Focus on: Other genes

## Foxo1

In [ ]:
gene="FOXO1"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

### Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_incl_zonation_normalized.csv',sep=';')

## Tek = Tie2

In [ ]:
gene="TEK"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

### Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_incl_zonation_normalized.csv',sep=';')

## Nos3 = eNOS

In [ ]:
gene="NOS3"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

### Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_incl_zonation_normalized.csv',sep=';')

## HTRA1

In [ ]:
gene="HTRA1"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

### Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_incl_zonation_normalized.csv',sep=';')

## EGFL8

In [ ]:
gene="EGFL8"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## FLT1

In [ ]:
gene="FLT1"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

### Incl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters2"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/incl_zonation/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_incl_zonation_normalized.csv',sep=';')

## KDR

In [ ]:
gene="KDR"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## NRP1

In [ ]:
gene="NRP1"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## NRP2

In [ ]:
gene="NRP2"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## EFNB2

In [ ]:
gene="EFNB2"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## ITGB1

In [ ]:
gene="ITGB1"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## ANGPT2

In [ ]:
gene="ANGPT2"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

## CDH5

In [ ]:
gene="CDH5"

### Plot

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

### Excl zonation

In [ ]:
# mean expression per group (normalized and scaled)
ex = sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", return_fig=True, layer="normalized").values_df
ex.columns = ['mean_expression']
ex

In [ ]:
# dotplot dot size = fraction of cells in group expressing Foxf2
fc = sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,7), standard_scale="var",return_fig=True, layer="normalized").dot_size_df
fc.columns = ['fraction_of_cells']
fc

In [ ]:
foxf2 = pd.concat([ex, fc], axis=1)
foxf2['gene'] = gene
foxf2['source'] = "Garcia2022"
foxf2['organism'] = "Human"
foxf2['cell_number'] = pd.DataFrame(adata.obs["clusters"].value_counts()).reindex(foxf2.index)
foxf2

In [ ]:
# export
name='Garcia2022'

In [ ]:
# Win
foxf2.to_csv(main_dir+'/Others_summarized/'+ date.today().strftime("%Y%m%d")+'_'+gene+'_'+name+'_normalized.csv',sep=';')

# Session Info

In [ ]:
sc.logging.print_versions()